In [13]:
from sepal_ui import widgetFactory as wf
from sepal_ui import widgetBinding as wb
from sepal_ui.scripts import mapping
from sepal_ui.scripts import utils as su

import ipyvuetify as v
from utils import messages as ms
from utils import parameters as pm
from scripts import vizualisation as viz
from functools import partial

import pandas as pd
import os
import geemap

In [14]:
#input and output as mutable objects 
class Viz_io:
    def __init__(self):
        #input
        self.sources = None
        self.bands = None
        
        #output

viz_io = Viz_io()

In [15]:
#pts map
pts_map = mapping.init_map()[1]

id_ = 'viz_widget'
title = 'Pts map'

viz_map = wf.Tile(
    id_,
    title,
    inputs=[pts_map]
) 

In [16]:
#captions
maps = [viz.setVizMap() for i in range(pm.getnbIntervals())]

title = 'Captions'

viz_captions = wf.Tile(
    id_,
    title
)

title_widget = viz_captions.children[0].children[0]
maps_layout = v.Layout(
    row=True,
    children=maps
)
viz_captions.children[0].children = [title_widget, maps_layout]

In [17]:
#output 
viz_output = wf.OutputWidget(ms.VIZ_INTRO)

#btn 
viz_btn = wf.ProcessBtn(ms.VIZ_BTN)

#create the inputs 
viz_sources = v.Select(
    items=[*pm.getSources()], 
    label=ms.VIZ_SOURCES, 
    v_model=None, 
    multiple=True
)

viz_bands = v.Select(
    items=[*pm.getAvailableBands()],
    label=ms.VIZ_BANDS,
    v_model=None
)

#bind the inputs
wb.bind(viz_sources, viz_io, 'sources', viz_output)
wb.bind(viz_bands, viz_io, 'bands', viz_output)

#create a band selector 
title = 'select your vizualisation Parameters'

viz_input = wf.Tile(
    id_,
    title,
    btn=viz_btn,
    inputs=[viz_sources, viz_bands],
    output=viz_output
)

In [18]:
#create a pts selector
viz_pts_slider = v.Slider(
    label= 'Pts selector',
    max=0,
    class_="mt-5", 
    thumb_label='always', 
    v_model=0
)

title = 'Pts selector'

viz_pts_selector = wf.Tile(
    id_,
    title,
    inputs=[viz_pts_slider]
)

In [19]:
#change the maps position when the slider move
def change_center(widget, event, data, m, maps, tile):
    
    pt = widget.v_model
    
    if not pt:
        return
    
    #load the pts 
    #pts = getattr(tb_io, 'pts')
    pts = pd.read_csv(os.path.expanduser('~') + '/test_pts.txt')
    lng = float(pts.loc[pt-1]['lng'])
    lat = float(pts.loc[pt-1]['lat'])
    
    #change the tile title 
    tile.children[0].children[0].children = [ms.VIZ_POINT.format(pt, lat, lng)]
    
    #change the center of all maps 
    m.setCenter(lng, lat, 8)
    [m.setCenter(lng, lat, 12) for m in maps]
    
    return 

viz_pts_slider.on_event('change', partial(
    change_center,
    m = pts_map,
    maps = maps,
    tile = viz_captions
))

In [20]:
#vizualise data 
from scripts import table as table_run

def process_start(widget, event, data, output, m, maps):
    
    #toggle the loading button
    su.toggleLoading(widget)
    
    #load the input 
    file = os.path.expanduser('~') + '/test_pts.txt'
    pts = pd.read_csv(file)
    #file = getattr(tb_io, 'file')
    #pts = getattr(tb_io, 'pts')
    bands = getattr(viz_io, 'bands')
    sources = getattr(viz_io, 'sources')
    
    #check input
    if not wb.checkInput(file, output, ms.NO_PTS): return su.toggleLoading(widget)
    #if not wb.checkInput(pts, output, ms.NO_PTS): return su.toggleLoading(widget)
    if not wb.checkInput(bands, output, ms.NO_BANDS): return su.toggleLoading(widget)
    if not wb.checkInput(sources, output, ms.NO_SOURCES): return su.toggleLoading(widget)
    if not len(sources) > 0: 
        su.displayIO(output, ms.NO_SOURCES, 'error')
        return su.toggleLoading(widget)
    if 'sentinel 2' in sources:
        su.displayIO(output, ms.SENTINEL_DISABLED, 'warning')
    
    #display points on the map
    pts = table_run.setMap(file, m) 
    
    #generat images (buffer at the moment)
    viz.setLayer(maps, pts, bands, sources)
    
    #change the slider size 
    viz_pts_slider.max = len(pts)
    
    #toggle the loading button
    su.toggleLoading(widget)
    
    return 

viz_btn.on_event('click', partial(
    process_start,
    output = viz_output,
    m = pts_map,
    maps = maps
))
    
    

In [21]:
viz_input

Layout(align_center=True, children=[Card(children=[Html(children=['select your vizualisation Parameters'], tag…

In [22]:
viz_pts_selector

Layout(align_center=True, children=[Card(children=[Html(children=['Pts selector'], tag='h2'), Flex(children=[L…

In [23]:
viz_captions

Layout(align_center=True, children=[Card(children=[Html(children=['Captions'], tag='h2'), Layout(children=[Map…

In [24]:
viz_map

Layout(align_center=True, children=[Card(children=[Html(children=['Pts map'], tag='h2'), Flex(children=[Layout…